In [2]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1715648571328_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1715648571328_0004,pyspark,idle,Link,Link,✔


In [3]:
from pyspark.sql import SparkSession
import sys

# Get Spark context
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sc.install_pypi_package("pandas==1.0.5","https://pypi.org/simple")
sc.install_pypi_package("scipy==1.4.1","https://pypi.org/simple")
sc.install_pypi_package("matplotlib==3.2.1","https://pypi.org/simple")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/af/f3/683bf2547a3eaeec15b39cef86f61e921b3b187f250fcd2b5c5fb4386369/pandas-1.0.5-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/dd/82/c1fe128f3526b128cfd185580ba40d01371c5d299fcf7f77968e22dfcc2e/scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/b2/c2/71fcf957710f3ba1f09088b35776a799ba7dd95f7c2b195ec800933b276b/matplotlib-3.2.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9d/ea/6d76df31432a0e6fdf81681a895f009a4bb47b3c39036db3e1b528191d52/pyparsing-3.1.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/f9/695d6bedebd747e5eb0fe8fad57b72fdf25411273a39791cde838d5a8f51/cycler-0.11.0-py3-none-any.whl
  Using cach

In [6]:
data = spark.read.parquet('s3://nyc-tlc/trip data/yellow_tripdata_2015*.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[VendorID: bigint, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: bigint, DOLocationID: bigint, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: int, airport_fee: int]

In [8]:
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 19
Total Rows: 146039231
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)

## Question 2:  Implementing a Reproducible Machine Learning Pipeline

In [9]:
from pyspark.sql.functions import concat_ws, col, hour
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Bucketizer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np
import pyspark.sql.functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### (a) Engineering 5 Features

#### 1. Categorical Feature: Rate Code (One-hot Encoded)
Applied one-hot encoding to Ratecode ID. 

In [10]:
ratecode_indexer = StringIndexer(inputCol="RatecodeID", outputCol="ratecode_indexed").setHandleInvalid("keep")
ratecode_encoder = OneHotEncoder(inputCols=["ratecode_indexed"], outputCols=["ratecode_encoded"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 2. Spatial Feature: Trip Zone Transition (Categorical, One-hot Encoded)
This feature considers both pickup and dropoff locations (using PULocationID and DOLocationID). Creating pairs of pick up and drop off locations to allow using trip transitions to predict tip amounts. 

In [11]:
# Create a combined column of pickup and dropoff location IDs then onehot encoding 
data = data.withColumn("zone_transition", concat_ws("_", "PULocationID", "DOLocationID"))
zone_transition_indexer = StringIndexer(inputCol="zone_transition", outputCol="zone_transition_indexed").setHandleInvalid("keep")
zone_transition_encoder = OneHotEncoder(inputCols=["zone_transition_indexed"], outputCols=["zone_transition_encoded"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 3. Spatial Feature: Trip Distance (Bucketized)
As taxis may see different traffic patterns and usage rates throughout the day, bucketizing the pickup hour can be very insightful.

In [12]:
## filtering out 0 distance trips and ultra long trips due to input errors
data = data.filter((F.col("trip_distance") > 0) & (F.col("trip_distance") <= 300))
data.printSchema()
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)
 |-- zone_transition: string (nullable = false)

Total Columns: 20
Total Rows: 145161756

In [13]:
splits = [-float('inf'), 1, 3, 5, 10, 20, float('inf')]

bucketizer = Bucketizer(splits=splits, inputCol="trip_distance", outputCol="trip_distance_buckets").setHandleInvalid("keep")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4. Floating Point Feature: Bank for the Buck (Floating Point)
It might be the case that if a passenger get to a destination with good value (spend relatively less money to get to the further location, he or she might tip higher. So I construct a bank for the buck feature. 

In [14]:
data = data.filter(F.col("fare_amount") > 0)
data.printSchema()
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)
 |-- zone_transition: string (nullable = false)

Total Columns: 20
Total Rows: 145098149

In [15]:
data = data.withColumn("bank_for_buck", col("trip_distance") / col("fare_amount"))
data.printSchema()
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)
 |-- zone_transition: string (nullable = false)
 |-- bank_for_buck: double (nullable = true)

Total Columns: 21
Total Rows: 145098149

#### 5.  Date and Time Feature: Hour of Day (Integer)
I am using `hour_of_day` transfrom from timestamp. It's an integer and pick up hour can imapact the tipping amount.

In [16]:
data = data.withColumn("hour_of_day", F.hour("tpep_pickup_datetime"))
data.printSchema()
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)
 |-- zone_transition: string (nullable = false)
 |-- bank_for_buck: double (nullable = true)
 |-- hour_of_day: integer (nullable = true)

Tot

### (b) Assembler, estimator, and pipeline

In [18]:
# Split the data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=0)

print("Training Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

# Persist data in memory to speed up I/O during training/testing
train_data.persist()
test_data.persist()

# Sample a fraction of the train_data
fraction = 0.01  # For example, 1% of the data
sampled_train_data = train_data.sample(False, fraction, seed=0)

sampled_train_data.persist()

# confirm they're cached
print(train_data.is_cached, test_data.is_cached)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Dataset Count: 116062438
Test Dataset Count: 29035711
True True

In [19]:
# confirm they're cached
print(sampled_train_data.is_cached)
print("Sampled Training Dataset Count: " + str(sampled_train_data.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True
Sampled Training Dataset Count: 1159356

In [23]:
# Final assembler with all features
finalAssembler = VectorAssembler(
    inputCols=["ratecode_encoded", 
               "zone_transition_encoded", 
               "trip_distance_buckets", 
               "bank_for_buck", 
               "hour_of_day"],
    outputCol="features",
    handleInvalid="keep"
)

# Estimator: Linear Regression
lr = LinearRegression(featuresCol='features', labelCol='tip_amount')

# Final pipeline
pipeline = Pipeline(stages=[
    ratecode_indexer,
    ratecode_encoder,
    zone_transition_indexer,
    zone_transition_encoder,
    bucketizer,
    finalAssembler,
    lr
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Question 3:  Finding Optimal Model Parameters 

In [24]:
# Setup the parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, np.arange(0, 0.1, 0.01).tolist()) \
    .addGrid(lr.elasticNetParam, [0, 1]) \
    .build()

evaluator = RegressionEvaluator(labelCol="tip_amount", predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  # Use 5 folds as per your requirement
start = time.time()
# Run the cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(sampled_train_data)
print(f"... completed job in {time.time() - start} seconds")
best_model = cvModel.bestModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

... completed job in 1848.2145175933838 seconds

In [26]:
# Use the best model found to make predictions on the test set
predictions = cvModel.transform(test_data)

# Evaluate the model on the test datathe
rmse = cvModel.getEvaluator().evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse:.3f}")

# Optionally, you might want to print out the best model's parameters
bestModel = cvModel.bestModel.stages[-1]  # Assuming Linear Regression is the last stage in the pipeline
print(f"Best Parameters: regParam = {bestModel._java_obj.getRegParam()}, elasticNetParam = {bestModel._java_obj.getElasticNetParam()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 2.114
Best Parameters: regParam = 0.01, elasticNetParam = 1.0

In [39]:
from pyspark.sql.functions import col, lit


# Filter out outliers
cleaned_data = data.filter((col("tip_amount") >= 0) & (col("tip_amount") <= 100))

print("Original Data Count: ", data.count())
print("Cleaned Data Count: ", cleaned_data.count())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Original Data Count:  145098149
Cleaned Data Count:  145096972

In [40]:
from pyspark.sql.functions import col, min, max, avg, stddev
tip_stats = cleaned_data.select(
    min(col("tip_amount")).alias("Min Tip"),
    max(col("tip_amount")).alias("Max Tip"),
    avg(col("tip_amount")).alias("Average Tip"),
    stddev(col("tip_amount")).alias("Standard Deviation of Tip")
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+------------------+-------------------------+
|Min Tip|Max Tip|       Average Tip|Standard Deviation of Tip|
+-------+-------+------------------+-------------------------+
|    0.0|  100.0|1.6949175623728914|        2.388542700697137|
+-------+-------+------------------+-------------------------+

In [29]:
coefficients = bestModel.coefficients
intercept = bestModel.intercept
print("Coefficients:", coefficients)
print("Intercept:", intercept)
trainingSummary = bestModel.summary
print("R2: ", trainingSummary.r2)
print("RMSE: ", trainingSummary.rootMeanSquaredError)
print("Residuals: ", trainingSummary.residuals().show())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object is not callable
Traceback (most recent call last):
TypeError: 'DataFrame' object is not callable



In [46]:
bestModel.summary.r2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.3736927313316317

In [42]:
# Assuming 'bestModel' is your trained LinearRegressionModel
coefficients = bestModel.coefficients  # This gets you the coefficients
features = ["ratecode_encoded", "trip_distance_buckets", "bank_for_buck", "hour_of_day","zone_transition_encoded"]  # Your feature names

# Printing feature names alongside their coefficients
print("Feature Coefficients:")
for feature, coeff in zip(features, coefficients):
    print(f"{feature}: {coeff}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Feature Coefficients:
ratecode_encoded: -3.0047144984985454
trip_distance_buckets: 0.0
bank_for_buck: 1.8279376453043643
hour_of_day: 1.7675936949463271
zone_transition_encoded: -0.5746666374487666

Report and interpret the optimal model’s predictive performance on your test
data. What are the most relevant features for predicting tip amount?
